In [ ]:
from hepmc.core.densities.qcd import ee_qq_ng, export_hepmc
from hepmc.core.densities.sarge import Sarge
from hepmc.core.phase_space.rambo import RamboOnDiet
from scipy.special import logit, expit
from hepmc.core.phase_space.mapping import MappedDensity
from hepmc.core.markov.metropolis import DefaultMetropolis
from hepmc.core.markov.base import MixingMarkovUpdate
from hepmc.core.markov.metropolis_adaptive import AdaptiveMetropolisUpdate
from hepmc.core.proposals import Gaussian
from hepmc import surrogate
from hepmc.core import densities
from hepmc.core.hamiltonian.hmc import HamiltonianUpdate
from hepmc.core.hamiltonian.wall_hmc import WallHMC
from hepmc.core.sampling import Sample
import numpy as np

In [ ]:
sarge = densities.Sarge(2, 3, 100.)

In [ ]:
eeqqg = ee_qq_ng(1, 100., 5., .3)

In [ ]:
rambo_mapping = RamboOnDiet(100., 3)
mapped = MappedDensity(eeqqg, rambo_mapping)

In [ ]:
%time sarge_sample = ImportanceMC(sarge)(eeqqg.pdf, 10000)

In [ ]:
print('estimated ESS:', util.effective_sample_size(sarge_sample, sarge_sample.mean, sarge_sample.variance))

In [ ]:
densities.export_hepmc(100., sarge_sample, "../samples/qcd/2-3/sarge_weighted.hepmc")

In [ ]:
bound = sarge_sample.weights.max()

In [ ]:
%%time
# acceptance-rejection sampler
sarge_sample_unweighted = np.empty((10000, 12))
for i in range(10000):
    aprob = 0
    while np.random.random() > aprob:
        proposal = sarge.proposal()
        f = eeqqg.pdf(proposal)
        g = sarge.proposal_pdf(None, proposal)
        aprob = f/(bound*g)
        if aprob > 1:
            print("bound too low")
    sarge_sample_unweighted[i] = proposal
sarge_sample_unweighted = Sample(data=sarge_sample_unweighted, weights=np.full(10000, 1./10000))

In [ ]:
densities.export_hepmc(100., sarge_sample_unweighted, "../samples/qcd/2-3/sarge_unweighted.hepmc")

In [ ]:
rambo_sample = PlainMC(5)(mapped, 10000)
rambo_sample.integral, rambo_sample.integral_err

In [ ]:
ys = mapped.pdf(rambo_sample.data)
max_y = ys.max()

In [ ]:
rambo = AcceptRejectSampler(mapped, max_y, 5)
%time rambo_sample = rambo.sample(1000)

In [ ]:
rambo_sample.data = rambo_mapping.map(rambo_sample.data)
rambo_sample.weights = np.full(rambo_sample.size, 1./rambo_sample.size)

In [ ]:
densities.export_hepmc(100., rambo_sample, "../samples/qcd/2-3/rambo.hepmc")

In [ ]:
# Metropolis with Rambo mapping
met = DefaultMetropolis(mapped, Gaussian(5, .01))
#start = met.sample(5000, np.random.rand(5)).data[-1] # warmup
met_sample = met.sample(10000, np.random.rand(5), burnin=5000, lag=2)

In [ ]:
met_sample.

In [ ]:
print('estimated ESS:', util.effective_sample_size(met_sample, met_sample.mean, met_sample.variance))

In [ ]:
met_sample.data = rambo_mapping.map(met_sample.data)
met_sample.weights = np.full(met_sample.size, 1./met_sample.size)
densities.export_hepmc(100., met_sample, "../samples/qcd/2-3/metropolis.hepmc")

In [ ]:
# adaptive Metropolis
def adapt_schedule(t):
    if t < 50000:
        return True
    else:
        return False
    
met = AdaptiveMetropolisUpdate(5, mapped, proposals.Gaussian(5, .01), t_initial=10000, adapt_schedule=adapt_schedule)
%time met_sample = met.sample(1060000, np.random.rand(5))
met_sample.data = met_sample.data[60000:] # discard warmup samples

In [ ]:
# thin sample
met_sample.data = met_sample.data[::100]

In [ ]:
met_sample.data = rambo_mapping.map(met_sample.data)
met_sample.weights = np.full(met_sample.size, 1./met_sample.size)
densities.export_hepmc(100., met_sample, "../samples/qcd/2-3/metropolis.hepmc")

In [ ]:
print('estimated ESS:', util.effective_sample_size(met_sample, met_sample.mean, met_sample.variance))

In [ ]:
# dual averaging Metropolis
def adapt_schedule(t):
    if t < 5000:
        return True
    else:
        return False
    
met = StochasticOptimizeUpdate(mapped, proposals.Gaussian(5, .01), adapt_schedule=adapt_schedule)
%time met_sample = met.sample(16000, np.random.rand(5))
met_sample.data = met_sample.data[6000:] # discard warmup samples

In [ ]:
print('estimated ESS:', util.effective_sample_size(met_sample, met_sample.mean, met_sample.variance))

In [ ]:
# now sarge + local in MC3
importance = DefaultMetropolis(3 * 4, eeqqg, proposal=sarge)  # using sarge, full momentum sample space
local = DefaultMetropolis(5, mapped, proposals.Gaussian(5, .01))  # using rambo, rambo sample space
update = MixingMarkovUpdate(5, [importance, local], target=mapped,
                            out_maps={0: rambo_mapping.map_inverse},
                            in_maps={0: rambo_mapping.map}) # rambo sample space, important to set target explicitly
start = update.sample(5000, np.random.rand(5)).data[-1] # warmup
%time mc3_sample = update.sample(10000, start)

In [ ]:
print('estimated ESS:', util.effective_sample_size(mc3_sample, mc3_sample.mean, mc3_sample.variance))

In [ ]:
mc3_sample.data = rambo_mapping.map(mc3_sample.data)
mc3_sample.weights = np.full(mc3_sample.size, 1./mc3_sample.size)
densities.export_hepmc(100., mc3_sample, "../samples/qcd/2-3/mc3.hepmc")